# Portfolio LONG ONLY

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

import random
import ast

import sys

In [2]:
data= pd.read_csv("../data/return.csv")
eonia= pd.read_csv("../data/eonia_rates.csv", sep= ";")
sp= pd.read_csv("../data/sp500.csv")
portfolio= pd.read_csv("../data/portfolio_constitution_short_long.csv")
return_predict_month= pd.read_excel("../data/return_predict_monthly.xlsx")

In [3]:
data2= data[[col for col in data.columns if col not in ["Date1", "Date2"]]]
data2= data2.T
# data2.loc['HES US Equity', :]
effectif_return= np.array([data2.loc[idx, :].values for idx in data2.index])[:, 14:]
effectif_return

array([[-1.966889e+00,  8.801784e-01, -8.061890e+00, ..., -3.543821e+00,
         8.713624e+00, -5.566540e+00],
       [-9.642124e-01,  7.383657e+00, -3.480635e+00, ..., -4.594867e+00,
         4.773207e+00, -3.347596e+00],
       [-6.008384e+00,  1.471754e+01, -2.014832e+00, ...,  8.690149e+00,
         9.703086e+00,  3.250417e+00],
       ...,
       [ 3.060476e+00,  5.227296e+00,  1.916458e+00, ...,  1.234873e-02,
         2.148413e+00,  6.946350e+00],
       [ 1.672717e+00,  5.677557e+00, -6.082109e-01, ..., -3.230081e+00,
        -2.426087e+00,  6.238303e-01],
       [-1.010077e+01, -7.559958e-01,  2.577451e+00, ..., -8.393829e-01,
        -1.876001e+00,  6.327681e+00]])

In [30]:
eonia_month= eonia[~eonia["mois"].isna()][["date", "mois"]]
eonia_month= eonia_month.sort_index(ascending= False)

sp["return"]= 100 * (sp["Close"].shift(-1) - sp["Close"])/sp["Close"]
sp["return"]= sp["return"].shift(1)
sp_return= sp[["Date", "return"]].dropna()
sp_return.head()

,Date,return
1,2010-01-01,-3.697425
2,2010-02-01,2.851369
3,2010-03-01,5.879643
4,2010-04-01,1.475923
5,2010-05-01,-8.197584


In [4]:
return_predict_month["Returns_predict"]= return_predict_month["Returns_predict"].apply(ast.literal_eval)
return_predict_month.head()

,Asset,Model,Returns_predict
0,HES US Equity,<catboost.core.CatBoostRegressor object at 0x7...,"[-2.832813166301726, 1.0798846506686033, -3.68..."
1,NEM US Equity,RandomForestRegressor,"[-0.9924100669999997, -0.8786505200000004, -4...."
2,INTC US Equity,RandomForestRegressor,"[-0.2536885459999996, -2.2266463759999984, 0.0..."
3,IFF US Equity,RandomForestRegressor,"[2.5741313260000025, 2.865933291000004, 1.6514..."
4,MOS US Equity,XGBRegressor,"[-0.2642870545387268, -8.006417274475098, 7.39..."


Le but va être d'être long de 20% des 5 titres avec le return prédit le plus fort par période, et d'être short de 20% pour ceux dont le return prédit est le plus faible

In [5]:
biggest_return= np.stack(np.array(return_predict_month["Returns_predict"]))


array([[15, 31,  4, ...,  4, 31, 16],
       [40, 20, 20, ..., 13, 13, 31],
       [21, 15, 40, ..., 31, 42, 42],
       ...,
       [ 5,  9,  8, ..., 25, 19, 26],
       [ 0,  4,  9, ..., 43,  3, 47],
       [47, 47, 31, ...,  3, 28,  3]], dtype=int64)

In [6]:
np.set_printoptions(threshold=sys.maxsize)

# sur internet
def top_return_col(K= 5, A= biggest_return):
    # Sort A, store the argsort for later usage
    sidx = np.argsort(A,axis=0)
    sA = A[sidx,np.arange(A.shape[1])]

    # Perform differentiation along rows and look for non-zero differentiations
    df = np.diff(sA,axis=0)!=0

    # Perform cumulative summation along rows from bottom upwards. 
    # Thus, summations < K should give us a mask of valid ones that are to 
    # be kept per column. Use this mask to set rest as zeros in sorted array.
    mask = (df[::-1].cumsum(0)<K)[::-1]
    sA[:-1] *=mask

    # Finally revert back to unsorted order by using sorted indices sidx
    out = sA[sidx.argsort(0),np.arange(sA.shape[1])]

    return out

top_return_col()

array([[-0.        ,  0.        , -0.        , -0.        , -0.        ,
        -0.        , -0.        , -0.        ,  0.        ,  0.        ,
        -0.        ,  0.        ,  9.96840533,  0.        , -0.        ,
        -0.        ,  0.        , -0.        ,  6.08717629,  6.4014399 ,
         0.        , -0.        ,  4.77267213,  0.        , -0.        ,
         0.        , -0.        , -0.        , -0.        ,  0.        ,
        -0.        , -0.        ,  0.        ,  0.        ,  0.        ,
        -0.        , -0.        ,  0.        , -0.        , -0.        ,
         0.        ,  0.        ,  0.        , -0.        , -0.        ,
        -0.        ,  0.        , -0.        ,  0.        ,  7.16149862,
        -0.        ,  0.        , -0.        , -0.        , -0.        ,
         0.        , -0.        ,  0.        ,  8.99827794,  0.        ,
        -0.        , -0.        ,  0.        , -0.        , -0.        ,
        -0.        , -0.        , -0.        , -0. 

Biggest returns

In [8]:
out= top_return_col()
indx_top_return= np.argwhere(out != 0)
indx_top_return= indx_top_return[np.argsort(indx_top_return[:, 1])]

In [10]:
top5_return_per_period_pred= biggest_return[
    indx_top_return[:, 0], indx_top_return[:, 1]
].reshape(biggest_return.shape[1], 5)
top5_return_per_period_pred

array([[ 5.75871134, 10.54684   ,  4.60980876,  7.48806   ,  5.36571074],
       [ 5.877584  ,  8.405854  ,  9.26572573, 17.53166008,  8.28258514],
       [ 6.83615347,  4.3113525 ,  7.39899683,  4.35756522,  5.02575159],
       [17.56224251,  8.06279741,  5.31098666,  5.877584  ,  4.00466681],
       [ 5.25563431,  4.8316289 ,  9.554813  ,  5.03561769,  5.6990912 ],
       [13.17699337,  6.87432408, 12.81032565,  6.41951008,  9.55473328],
       [ 5.35884079,  5.53086641,  5.29644251,  8.1274358 ,  5.7570854 ],
       [ 2.13009682,  9.29113674,  4.81652683, 10.40649507,  3.97906017],
       [ 7.188264  ,  9.13400163, 30.35089308,  8.32238579, 16.45786487],
       [11.08312607, 19.14242554, 10.54684   , 11.42222023, 11.22937489],
       [ 6.99267974,  3.3414715 ,  4.05371182, 16.34384695,  3.371494  ],
       [ 6.97224515,  5.95819851,  5.44288356,  6.37983036,  8.89245605],
       [ 6.73210322,  5.53288174,  9.96840533, 10.16592   ,  7.19326878],
       [ 8.95014954,  7.14001023,  6.1

In [11]:
top5_return_per_period_eff= effectif_return[
    indx_top_return[:, 0], indx_top_return[:, 1]
].reshape(effectif_return.shape[1], 5)

top5_return_per_period_eff

array([[ 5.272747e+00,  8.405854e+00,  2.555541e+00,  6.806419e+00,
         6.857790e+00],
       [ 7.105256e+00,  9.633279e+00,  1.351057e+00,  5.813953e+00,
         7.984293e+00],
       [ 1.016986e+01, -1.288443e+00, -5.356666e+00, -2.348420e+00,
        -1.188119e+00],
       [-1.947005e+01, -7.229340e+00,  7.189656e-01,  7.081340e-01,
         3.066477e+00],
       [-1.759524e+01, -4.778954e+00,  1.343125e+00, -1.559333e+01,
        -1.839080e+00],
       [-1.434931e+01, -2.072801e+00,  3.384030e+00,  4.627892e-01,
         6.528044e-01],
       [ 1.466212e-01,  1.404494e-01, -2.562226e+01, -1.474891e+01,
         5.986090e+00],
       [-3.392978e+00,  1.476378e+01,  5.581289e+00,  6.163622e+00,
         2.760164e+00],
       [ 4.489019e+00,  2.895594e+00,  1.039516e+01, -4.961210e+00,
         2.696918e+00],
       [-7.105893e-01, -1.686747e+00, -8.626596e+00, -7.657394e+00,
        -4.905309e+00],
       [-9.058468e-01, -8.277611e+00,  1.590019e+01,  6.653247e+00,
         1.8

Lowest returns

In [15]:
top_lose= -top_return_col(K= 5, A= -1*biggest_return)

indx_low_return= np.argwhere(top_lose != 0)
indx_low_return= indx_low_return[np.argsort(indx_low_return[:, 1])]
indx_low_return

array([[  0,   0],
       [  5,   0],
       [ 31,   0],
       [  9,   0],
       [ 47,   0],
       [ 47,   1],
       [ 21,   1],
       [  4,   1],
       [  9,   1],
       [ 40,   1],
       [  9,   2],
       [ 31,   2],
       [  8,   2],
       [ 15,   2],
       [  1,   2],
       [  4,   3],
       [ 28,   3],
       [ 25,   3],
       [ 40,   3],
       [ 15,   3],
       [  1,   4],
       [  4,   4],
       [ 47,   4],
       [ 31,   4],
       [ 41,   4],
       [ 37,   5],
       [  2,   5],
       [ 41,   5],
       [ 40,   5],
       [ 31,   5],
       [ 45,   6],
       [ 37,   6],
       [ 28,   6],
       [  0,   6],
       [ 43,   6],
       [ 18,   7],
       [ 43,   7],
       [  4,   7],
       [ 28,   7],
       [ 41,   7],
       [ 16,   8],
       [ 21,   8],
       [ 31,   8],
       [ 28,   8],
       [ 34,   8],
       [  3,   9],
       [ 24,   9],
       [ 31,   9],
       [ 22,   9],
       [ 41,   9],
       [ 16,  10],
       [ 28,  10],
       [ 41,

In [17]:
lose5_return_per_period_pred= biggest_return[
    indx_low_return[:, 0], indx_low_return[:, 1]
].reshape(biggest_return.shape[1], 5)
lose5_return_per_period_pred

array([[ -2.83281317,  -1.632124  ,  -1.07677519,  -1.08697921,
         -5.39908123],
       [ -9.02018261,  -2.29913163,  -8.00641727,  -3.76748991,
         -2.37952709],
       [ -7.13963379, -17.54606247,  -4.90536492,  -3.754204  ,
         -4.88805056],
       [ -5.02691555,  -7.49933863,  -3.69499876,  -4.38339758,
         -4.19241   ],
       [ -3.28826456,  -5.16664171,  -3.65749121,  -2.38211632,
         -8.42711408],
       [ -5.31940829,  -4.65649521,  -9.42161346,  -7.19674587,
        -11.98221493],
       [ -9.590208  ,  -8.41057623,  -8.6257019 , -11.36244174,
         -9.96574386],
       [-13.18205643, -13.46047441, -26.25718307, -22.13038254,
        -12.42180918],
       [ -7.41744082,  -3.15190983, -11.57111645,  -4.31038857,
         -1.78382757],
       [ -4.09589309,  -7.00855741,  -4.22250557,  -3.10862432,
         -3.81427775],
       [ -3.78518157,  -6.64799976,  -8.19638546, -16.54318428,
         -4.55259991],
       [ -4.74912796, -16.65472243,  -1.849

In [18]:
lose5_return_per_period_eff= effectif_return[
    indx_low_return[:, 0], indx_low_return[:, 1]
].reshape(effectif_return.shape[1], 5)

lose5_return_per_period_eff

array([[-1.966889e+00, -1.245211e+00, -6.623236e+00, -9.302326e-01,
        -1.010077e+01],
       [-7.559958e-01,  3.410427e+00, -4.876953e+00,  3.755868e+00,
         4.533223e+00],
       [-3.555268e+00, -4.615385e+00,  4.604569e+00, -1.221889e+01,
        -3.480635e+00],
       [-4.403670e+00, -2.933608e+00,  2.209794e+00, -8.744037e+00,
        -1.662866e+00],
       [ 3.038725e+00,  4.414587e+00, -1.150678e+01,  5.007153e+00,
         6.642778e+00],
       [-1.113668e+01, -8.984219e+00, -2.084767e+01, -1.044896e+01,
        -6.948229e+00],
       [-1.139199e+01, -1.987325e+01, -2.256763e+01, -1.144987e+01,
        -1.712123e+01],
       [ 1.466511e+01,  2.998601e+01,  1.958342e+01,  2.472496e+01,
         2.565547e+01],
       [-5.911718e+00,  5.906389e+00, -2.401372e+00, -6.176190e-02,
         1.036269e+00],
       [-2.827392e+00,  4.347826e+00, -5.096661e+00, -9.318996e+00,
        -1.093206e+01],
       [ 1.042307e+01,  2.856676e+01,  1.007401e+00,  2.425926e+01,
        -7.4

In [28]:
pred_lose_= -lose5_return_per_period_pred/5
eff_lose_= -lose5_return_per_period_eff/5

pred_top_= top5_return_per_period_pred/5
eff_top_= top5_return_per_period_eff/5

pred_portfolio= pred_lose_ + pred_top_
pred_portfolio

eff_portfolio= eff_lose_ + eff_top_
eff_total_period= np.sum(eff_portfolio, axis= 1)

pred_portfolio= pred_lose_ + pred_top_
pred_total_period= np.sum(pred_portfolio, axis= 1)

In [39]:
# get dates
date= sp_return.iloc[14:,0].values

In [46]:
date

array(['2011-03-01', '2011-04-01', '2011-05-01', '2011-06-01',
       '2011-07-01', '2011-08-01', '2011-09-01', '2011-10-01',
       '2011-11-01', '2011-12-01', '2012-01-01', '2012-02-01',
       '2012-03-01', '2012-04-01', '2012-05-01', '2012-06-01',
       '2012-07-01', '2012-08-01', '2012-09-01', '2012-10-01',
       '2012-11-01', '2012-12-01', '2013-01-01', '2013-02-01',
       '2013-03-01', '2013-04-01', '2013-05-01', '2013-06-01',
       '2013-07-01', '2013-08-01', '2013-09-01', '2013-10-01',
       '2013-11-01', '2013-12-01', '2014-01-01', '2014-02-01',
       '2014-03-01', '2014-04-01', '2014-05-01', '2014-06-01',
       '2014-07-01', '2014-08-01', '2014-09-01', '2014-10-01',
       '2014-11-01', '2014-12-01', '2015-01-01', '2015-02-01',
       '2015-03-01', '2015-04-01', '2015-05-01', '2015-06-01',
       '2015-07-01', '2015-08-01', '2015-09-01', '2015-10-01',
       '2015-11-01', '2015-12-01', '2016-01-01', '2016-02-01',
       '2016-03-01', '2016-04-01', '2016-05-01', '2016-

In [52]:
short_long_portfolio_strat= pd.DataFrame({
    "date": date
    , "effectif": eff_total_period
    , "pred": pred_total_period
    , "eonia": eonia_month.iloc[14:,1].values
})
short_long_portfolio_strat.to_csv("../data/long_short_return_.csv", index= False, header= True)

In [51]:
short_long_portfolio_strat

,date,effectif,pred,eonia
0,2011-03-01,10.152938,9.159381,"0,659"
1,2011-04-01,5.164254,14.967231,"0,966"
2,2011-05-01,3.850764,13.232627,"1,033"
3,2011-06-01,-1.334285,13.123068,"1,124"
4,2011-07-01,-9.211988,10.659683,"1,012"
5,2011-08-01,9.288654,17.482473,"0,906"
6,2011-09-01,9.661192,15.605069,"1,005"
7,2011-10-01,-17.747819,23.615044,"0,96"
8,2011-11-01,3.389535,19.937619,"0,79"
9,2011-12-01,0.048130,17.134769,"0,627"
